In [ ]:
import pandas as pd
import openai
import time
import re

In [ ]:
# df = pd.read_csv('data oca awal.csv')
df=pd.read_excel("Netmonk HI Data.xlsx")
df

In [ ]:
# df = df.iloc[6000:7500].reset_index(drop=True)
# df

In [ ]:
import pathlib
import textwrap

import google.generativeai as genai

from IPython.display import display
from IPython.display import Markdown
def to_markdown(text):
    text = text.replace('*', ' *')
    return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [ ]:
genai.configure(api_key = "[api key]") 
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)


In [ ]:
import time
import pandas as pd
import google.generativeai as genai

def clean_output(pertanyaan, jawaban):
    # Hapus strip dari pertanyaan
    pertanyaan = pertanyaan.replace("-", "").replace("*", "").strip()
    
    # Pisahkan jawaban menjadi paragraf-paragraf
    jawaban_paragraf = jawaban.split("\n")
    
    # Ambil hanya paragraf pertama
    jawaban_cleaned = []
    for line in jawaban_paragraf:
        if line.strip() == "":
            break
        jawaban_cleaned.append(line.strip())
    
    jawaban_cleaned = " ".join(jawaban_cleaned)
    
    return pertanyaan, jawaban_cleaned

def extract_customer_agent(text):
    prompt = f"""
    Anda adalah ahli bahasa yang mengevaluasi percakapan layanan pelanggan. 
    Analisis percakapan dan ekstrak pertanyaan dan jawaban dari percakapan berikut '{text}' yang:
    1. Pertanyaan berkaitan dengan produk, fitur, kebijakan, informasi layanan, atau sistem secara umum
    2. Pertanyaan bisa dijawab oleh AI/chatbot tanpa perlu informasi spesifik pelanggan
    3. Pertanyaan tidak menanyakan data real-time, status pesanan, tiket, status email masuk, status gangguan aplikasi, atau akses ke sistem internal.
    4. Jawaban yang diberikan agent sesuai untuk pertanyaan tersebut
    5. Pertanyaan berkaitan dengan cara melakukan sesuatu atau prosedur
    7. Pertanyaan tidak memerlukan jawaban/tindakan manual oleh agen/manusia
    8. Jawaban tidak membutuhkan data pribadi customer. Seperti meminta mengisi data diri, nomor telepon, email, alamat, dll. 
    9. Jawaban tidak meminta customer mununggu untuk info/proses lebih lanjut atau menunggu balasan dari pihak lain atau melakukan pengecekan atau meminta konfirmasi.
    10. Pertanyaan dan jawabannya tidak mengandung perihal nomor tiket (DOA) atau invoice (INV) atau nomor pesanan (PO) atau nomor pelanggan (CIF) atau LOP atau nomor lainnya.
    11. Pertanyaan tidak menanyakan hal mengenai status diri sendiri.
    12. Jawaban tidak meminta capture atau screenshoot atau meminta informasi yang hanya bisa dilihat di layar customer
    13. Jawaban tidak memberikan pernyataan bahwa agent telah melakukan tindakan tertentu atau telah menerima informasi tertentu dari customer.
    14. Pertanyaan hanya diambil dari customer saja dan tidak menanyakan keadaan atau konifrmasi seperti "Apakah informasi yang diberikan sudah cukup jelas?" 
    
    Berikan hasil dalam format berikut:
    - **Pertanyaan**: [Tuliskan pertanyaan dari customer yang sudah dirapikan dalam bentuk kalimat tanya yang jelas]
    - **Jawaban**: [Tuliskan jawaban dari agen yang sudah dirapikan menjadi kalimat yang SINGKAT, padat, jelas, dan mudah dipahami tanpa menyebut nama customer]

    Jika percakapan tidak berisi pertanyaan atau jawaban yang jelas, berikan "No data".
    Jika terdapat nama orang pada jawaban, hapus nama dan ganti/sesuaikan dengan kalimatnya saja.
    Jika pertanyaan adalah tata cara berikan secara step by step tetapi tetap singkat dan jelas.
    """
    try:
        model = genai.GenerativeModel('gemini-2.0-flash')  #15RPM #1500RPD
        response = model.generate_content(prompt)
        output = response.text.strip()
        print(f"Output: {output}")

        pertanyaan, jawaban = "No data", "No data"
        if "**Pertanyaan**:" in output and "**Jawaban**:" in output:
            parts = output.split("**Jawaban**:")
            pertanyaan = parts[0].replace("**Pertanyaan**:", "").strip()
            jawaban = parts[1].strip() if len(parts) > 1 else "No data"
        
        # Cleaning the output
        pertanyaan, jawaban = clean_output(pertanyaan, jawaban)

        return pertanyaan, jawaban
    except Exception as e:
        print(f"Error: {e}")
        return "Error", "Error"

def analyze_conversations(df, batch_size=15, sleep_time=60):
    total_rows = len(df)
    
    # Menambahkan kolom baru untuk hasil ekstraksi
    if 'Pertanyaan' not in df.columns:
        df['Pertanyaan'] = ""
    if 'Jawaban' not in df.columns:
        df['Jawaban'] = ""

    for i in range(total_rows):
        try:
            text = df.iloc[i]['Conversation']
            
            if pd.isna(text) or text.strip() == "":
                df.loc[i, 'Pertanyaan'] = "No data"
                df.loc[i, 'Jawaban'] = "No data"
                continue
            
            # Ekstraksi pertanyaan customer dan jawaban agen
            pertanyaan, jawaban = extract_customer_agent(text)
            df.loc[i, 'Pertanyaan'] = pertanyaan
            df.loc[i, 'Jawaban'] = jawaban
            
        except Exception as e:
            print(f"[{i}]> Error: {e}")
            df.loc[i, 'Pertanyaan'] = "Error"
            df.loc[i, 'Jawaban'] = "Error"
        
        # Batch processing untuk menghindari batasan API
        if (i + 1) % batch_size == 0:
            print(f"Processed {i + 1}/{total_rows} rows. Sleeping for {sleep_time} seconds...")
            time.sleep(sleep_time)
    
    return df

In [ ]:
%%time
analyze_conversations(df)
df

In [ ]:
df.to_csv('padi2024_part5.csv', index=False)

In [ ]:
import os
def gabungkan_csv(folder_path):
    files = [pd.read_csv(os.path.join(folder_path, file)) for file in os.listdir(folder_path) if file.endswith('.csv')]
    return pd.concat(files, ignore_index=True)


# Contoh penggunaan
folder_path = '/Users/gibranariq/Documents/project/DDB Telkom/mytens'  # Ganti dengan path folder kamu
df = gabungkan_csv(folder_path)

# Simpan hasil gabungan menjadi file baru
df.to_csv('mytens_full.csv', index=False)

print('Penggabungan berhasil! File disimpan sebagai hasil_gabungan.csv')

In [ ]:
df

In [ ]:
# Filter the DataFrame for rows where 'Pertanyaan' or 'Jawaban' is "Error"
error_rows = df[(df['Pertanyaan'] == "Error") | (df['Jawaban'] == "Error")]
    
    # Display the filtered DataFrame
error_rows

In [ ]:
import time
import pandas as pd
import google.generativeai as genai

def clean_output(pertanyaan, jawaban):
    # Hapus strip dari pertanyaan
    pertanyaan = pertanyaan.replace("-", "").replace("*", "").strip()
    
    # Pisahkan jawaban menjadi paragraf-paragraf
    jawaban_paragraf = jawaban.split("\n")
    
    # Ambil hanya paragraf pertama
    jawaban_cleaned = []
    for line in jawaban_paragraf:
        if line.strip() == "":
            break
        jawaban_cleaned.append(line.strip())
    
    jawaban_cleaned = " ".join(jawaban_cleaned)
    
    return pertanyaan, jawaban_cleaned

def extract_customer_agent(text):
    prompt = f"""
    Anda adalah ahli bahasa yang mengevaluasi percakapan layanan pelanggan. 
    Analisis percakapan dan ekstrak pertanyaan dan jawaban dari percakapan berikut '{text}' yang:
    1. Pertanyaan berkaitan dengan produk, fitur, kebijakan, informasi layanan, atau sistem secara umum
    2. Pertanyaan bisa dijawab oleh AI/chatbot tanpa perlu informasi spesifik pelanggan
    3. Pertanyaan tidak menanyakan data real-time, status pesanan, tiket, status email masuk, status gangguan aplikasi, atau akses ke sistem internal.
    4. Jawaban yang diberikan agent sesuai untuk pertanyaan tersebut
    5. Pertanyaan berkaitan dengan cara melakukan sesuatu atau prosedur
    7. Pertanyaan tidak memerlukan jawaban/tindakan manual oleh agen/manusia
    8. Jawaban tidak membutuhkan data pribadi customer. Seperti meminta mengisi data diri, nomor telepon, email, alamat, dll. 
    9. Jawaban tidak meminta customer mununggu untuk info/proses lebih lanjut atau menunggu balasan dari pihak lain atau melakukan pengecekan atau meminta konfirmasi.
    10. Pertanyaan dan jawabannya tidak mengandung perihal nomor tiket (DOA) atau invoice (INV) atau nomor pesanan (PO) atau nomor pelanggan (CIF) atau LOP atau nomor lainnya.
    11. Pertanyaan tidak menanyakan hal mengenai status diri sendiri.
    12. Jawaban tidak meminta capture atau screenshoot atau meminta informasi yang hanya bisa dilihat di layar customer
    13. Jawaban tidak memberikan pernyataan bahwa agent telah melakukan tindakan tertentu atau telah menerima informasi tertentu dari customer.
    14. Pertanyaan hanya diambil dari customer saja dan tidak menanyakan keadaan atau konifrmasi seperti "Apakah informasi yang diberikan sudah cukup jelas?" 
    
    Berikan hasil dalam format berikut:
    - **Pertanyaan**: [Tuliskan pertanyaan dari customer yang sudah dirapikan dalam bentuk kalimat tanya yang jelas]
    - **Jawaban**: [Tuliskan jawaban dari agen yang sudah dirapikan menjadi kalimat yang SINGKAT, padat, jelas, dan mudah dipahami tanpa menyebut nama customer]

    Jika percakapan tidak berisi pertanyaan atau jawaban yang jelas, berikan "No data".
    Jika terdapat nama orang pada jawaban, hapus nama dan ganti/sesuaikan dengan kalimatnya saja.
    Jika pertanyaan adalah tata cara berikan secara step by step tetapi tetap singkat dan jelas.
    """
    try:
        model = genai.GenerativeModel('gemini-2.0-flash')  #15RPM #1500RPD
        response = model.generate_content(prompt)
        output = response.text.strip()
        print(f"Output: {output}")

        pertanyaan, jawaban = "No data", "No data"
        if "**Pertanyaan**:" in output and "**Jawaban**:" in output:
            parts = output.split("**Jawaban**:")
            pertanyaan = parts[0].replace("**Pertanyaan**:", "").strip()
            jawaban = parts[1].strip() if len(parts) > 1 else "No data"
        
        # Cleaning the output
        pertanyaan, jawaban = clean_output(pertanyaan, jawaban)

        return pertanyaan, jawaban
    except Exception as e:
        print(f"Error: {e}")
        return "Error", "Error"

def analyze_conversations(df, batch_size=15, sleep_time=60):
    total_rows = len(df)
    
    # Menambahkan kolom baru untuk hasil ekstraksi
    if 'Pertanyaan' not in df.columns:
        df['Pertanyaan'] = ""
    if 'Jawaban' not in df.columns:
        df['Jawaban'] = ""

    for i in range(total_rows):
        try:
            text = df.iloc[i]['Conversation']
            
            if pd.isna(text) or text.strip() == "":
                df.loc[i, 'Pertanyaan'] = "No data"
                df.loc[i, 'Jawaban'] = "No data"
                continue
            
            # Ekstraksi pertanyaan customer dan jawaban agen
            pertanyaan, jawaban = extract_customer_agent(text)
            df.loc[i, 'Pertanyaan'] = pertanyaan
            df.loc[i, 'Jawaban'] = jawaban
            
        except Exception as e:
            print(f"[{i}]> Error: {e}")
            df.loc[i, 'Pertanyaan'] = "Error"
            df.loc[i, 'Jawaban'] = "Error"
        
        # Batch processing untuk menghindari batasan API
        if (i + 1) % batch_size == 0:
            print(f"Processed {i + 1}/{total_rows} rows. Sleeping for {sleep_time} seconds...")
            time.sleep(sleep_time)
    
    return df

def reprocess_errors(df, batch_size=15, sleep_time=60):
    error_indices = df[(df['Pertanyaan'] == "Error") | (df['Jawaban'] == "Error")].index
    total_errors = len(error_indices)

    for count, i in enumerate(error_indices):
        try:
            text = df.loc[i, 'Conversation']
            if pd.isna(text) or text.strip() == "":
                df.loc[i, 'Pertanyaan'] = "No data"
                df.loc[i, 'Jawaban'] = "No data"
                continue

            pertanyaan, jawaban = extract_customer_agent(text)
            df.loc[i, 'Pertanyaan'] = pertanyaan
            df.loc[i, 'Jawaban'] = jawaban

        except Exception as e:
            print(f"Reprocessing Error at row {i}: {e}")
            df.loc[i, 'Pertanyaan'] = "Error"
            df.loc[i, 'Jawaban'] = "Error"

        if (count + 1) % batch_size == 0:
            print(f"Processed {count + 1}/{total_errors} errors. Sleeping for {sleep_time} seconds...")
            time.sleep(sleep_time)
    return df

In [ ]:
%%time
df = reprocess_errors(df)
df

In [ ]:
def clean_text_pertanyaan(text):
    # Pisahkan teks menjadi paragraf berdasarkan newline ganda dan ambil paragraf terakhir
    paragraphs = text.strip().split('\n\n')
    return paragraphs[-1].strip()

# Terapkan fungsi untuk mendapatkan paragraf terakhir ke kolom 'Pertanyaan'
df['cleaned_pertanyaan'] = df['Pertanyaan'].apply(clean_text_pertanyaan)

df

In [ ]:
df["cleaned_pertanyaan"]


In [ ]:
# Menghapus baris yang berisi "No Data"
df = df[(df['cleaned_pertanyaan'] != 'No data') & (df['Jawaban'] != 'No data')]

# Reset indeks
df = df.reset_index(drop=True)

# Menampilkan dataframe yang telah dibersihkan
df

In [ ]:
df.to_csv('Netmonk HI_full_new.csv', index=False)